In [1]:
import pandas as pd
import numpy as np

In [2]:
df_1 = pd.read_csv('./data/sample_submission.csv')
df_1.head()

,ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer
0,1,1,0,0,0,0
1,2,1,0,0,0,0
2,3,1,0,0,0,0
3,4,1,0,0,0,0
4,5,1,0,0,0,0


In [7]:
df_1.columns

Index(['ID', 'Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'], dtype='object')

In [3]:
df_1.sum().index

Index(['ID', 'Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'], dtype='object')

In [13]:
df_test = pd.read_csv('./data/test.csv')
df_test.sample(5)

,ID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
8738,8739,Blackie,2014-10-27 14:04:00,Dog,Intact Male,1 month,Irish Setter/Pit Bull,Black
8879,8880,Eleanor,2015-10-21 13:40:00,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby
6379,6380,Sputnik,2015-10-18 13:42:00,Cat,Neutered Male,7 months,Domestic Shorthair Mix,Black
6708,6709,Simon,2016-02-12 00:00:00,Cat,Neutered Male,10 years,Siamese Mix,Seal Point
6584,6585,NaN,2015-07-18 12:23:00,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Brown Tabby/White


In [15]:
df_test.shape

(11456, 8)

In [16]:
df_train = pd.read_csv('./data/train.csv')
df_train.sample(5)

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
5679,A675600,Lucy,2014-03-31 19:01:00,Return_to_owner,NaN,Dog,Spayed Female,5 years,Beagle Mix,Tricolor
8046,A675886,Jirah,2014-04-02 15:48:00,Return_to_owner,NaN,Dog,Spayed Female,8 years,Labrador Retriever Mix,Red/White
20436,A667374,Rascal,2013-11-20 17:12:00,Return_to_owner,NaN,Dog,Neutered Male,10 months,Chihuahua Shorthair Mix,Chocolate/White
11465,A696145,Bubba,2015-03-15 15:53:00,Adoption,NaN,Dog,Neutered Male,3 years,German Shepherd Mix,Red/Black
4010,A711011,NaN,2015-08-31 19:24:00,Transfer,Partner,Cat,Unknown,3 days,Domestic Shorthair Mix,Brown Tabby


In [17]:
df_train.shape

(26729, 10)

# Search duplicates in train dataset

In [23]:
df_train.shape

(26729, 10)

In [ ]:
# find current duplicates in train
df_train.duplicated().sum()

np.int64(0)

In [ ]:
# find current duplicates in test
df_test.duplicated().sum()

np.int64(0)

In [34]:
df_train.columns

Index(['AnimalID', 'Name', 'DateTime', 'OutcomeType', 'OutcomeSubtype',
       'AnimalType', 'SexuponOutcome', 'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [42]:
df_test.columns

Index(['ID', 'Name', 'DateTime', 'AnimalType', 'SexuponOutcome',
       'AgeuponOutcome', 'Breed', 'Color'],
      dtype='object')

In [36]:
# drop target columns from train.csv so we can find duplicated rows
df_train_2 = df_train.drop(['OutcomeType', 'OutcomeSubtype'], axis=1)

In [45]:
# find if column id matches between dataframes
df_test['ID'].isin(df_train['AnimalID']).sum()

np.int64(0)

In [46]:
# rename id col for matching
df_test = df_test.rename(columns={"ID": 'AnimalID'})

In [47]:
# we can now merge both dataframes so we get more data
df = pd.concat([df_train_2, df_test])
df.shape

(38185, 8)

In [38]:
# find duplicates between dataframes
df.duplicated().sum()

np.int64(0)

# 1. EDA

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38185 entries, 0 to 11455
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AnimalID        26729 non-null  object 
 1   Name            27269 non-null  object 
 2   DateTime        38185 non-null  object 
 3   AnimalType      38185 non-null  object 
 4   SexuponOutcome  38184 non-null  object 
 5   AgeuponOutcome  38161 non-null  object 
 6   Breed           38185 non-null  object 
 7   Color           38185 non-null  object 
 8   ID              11456 non-null  float64
dtypes: float64(1), object(8)
memory usage: 2.9+ MB


In [50]:
df.sample(10)

,AnimalID,Name,DateTime,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
209,A714575,Isis,2015-10-31 15:18:00,Cat,Intact Female,15 years,Domestic Shorthair Mix,Black
11222,A686309,Paige,2014-08-21 14:23:00,Cat,Spayed Female,2 months,Domestic Shorthair Mix,Torbie
10269,A718235,Graziella,2015-12-27 18:22:00,Dog,Spayed Female,15 years,American Eskimo,White
2168,2169,Ernestine,2016-02-02 15:22:00,Cat,Spayed Female,4 months,Domestic Shorthair Mix,Cream Tabby/White
15650,A686534,NaN,2014-08-24 09:00:00,Cat,Intact Female,8 months,Domestic Shorthair Mix,Black/White
10818,10819,Sadie,2014-08-09 17:57:00,Dog,Spayed Female,1 year,German Shepherd/Labrador Retriever,Black/Brown
25715,A697897,NaN,2015-03-03 09:31:00,Cat,Intact Male,3 years,Domestic Longhair Mix,Brown Tabby
7754,A664629,NaN,2013-10-06 15:45:00,Cat,Intact Male,1 weeks,Domestic Medium Hair Mix,Orange Tabby
6087,6088,Charlie,2015-12-28 12:04:00,Dog,Intact Male,3 years,Chihuahua Shorthair/Whippet,Brown
10978,A699378,NaN,2015-03-30 16:44:00,Dog,Spayed Female,2 months,Black Mouth Cur Mix,Brown/White
